In [ ]:
# Kaggleのノートブック環境外でモデルをトレーニングし、トレーニングされたモデルを外部データソースとしてアップロードしてKaggleに送信する

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import cv2
plt.style.use('ggplot')
from IPython.display import Video
from IPython.display import HTML
import os

In [ ]:
# ひとまず何があるか確認
listdir = "../input/deepfake-detection-challenge/"
os.listdir("../input/deepfake-detection-challenge")

In [ ]:
# train_sample_videos と test_videos には動画ファイルが入ってる
os.listdir("../input/deepfake-detection-challenge/train_sample_videos")[:5]

In [ ]:
pd.read_csv(listdir + "sample_submission.csv").head(5)

In [ ]:
# もちろんだがtestデータとtrainデータには同じ動画ファイルはないようだ
"""
for sample_id in pd.read_csv(listdir + "sample_submission.csv")["filename"].values:
    if sample_id in os.listdir("../input/deepfake-detection-challenge/train_sample_videos"):
        print(sample_id)
    else:
        pass
"""

In [ ]:
# trainデータの中身はこんな感じ
train_sample_metadata = pd.read_json('../input/deepfake-detection-challenge/train_sample_videos/metadata.json').T
train_sample_metadata.head()

In [ ]:
print("labelについて")
display(train_sample_metadata["label"].value_counts())
print("splitについて")
display(train_sample_metadata["split"].value_counts())
print("original")
display(train_sample_metadata["original"].value_counts())

In [ ]:
# いったんどんなんか見てみる
import cv2 as cv
import os
import matplotlib.pylab as plt
train_dir = '/kaggle/input/deepfake-detection-challenge/train_sample_videos/'
fig, ax = plt.subplots(1,1, figsize=(15, 15))
train_video_files = [train_dir + x for x in os.listdir(train_dir)]
video_file = '/kaggle/input/deepfake-detection-challenge/train_sample_videos/akxoopqjqz.mp4'
cap = cv.VideoCapture(video_file)
success, image = cap.read()
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
cap.release()   
ax.imshow(image)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.title.set_text(f"FRAME 0: {video_file.split('/')[-1]}")
plt.grid(False)

In [ ]:
# 動画を再生してみる。とにかく違和感がすごい。
fake_videos = list(train_sample_metadata.loc[train_sample_metadata.label=='FAKE'].index)
from base64 import b64encode

def play_video(video_file, subset='train_sample_videos'):
    video_url = open(os.path.join('../input/deepfake-detection-challenge', subset,video_file),'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(video_url).decode()
    return HTML("""<video width=500 controls><source src="%s" type="video/mp4"></video>""" % data_url)
play_video(fake_videos[0])

In [ ]:
#モデル作製の流れは
#①顔だけに注目した画像を切り抜く。
#②YOLOなどでモデル作製でいいのか？？？？